<a href="https://colab.research.google.com/github/GabriRH21/Recomendador-Turistico/blob/main/Clasificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entrenamiento para clasificador**

## Importacion del librerias:

In [ ]:
import openpyxl
import nltk
from bs4 import BeautifulSoup
import re
import math
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Declaración de metodos que usaremos:

In [ ]:
def get_description(inf):
  if inf is None:
    return ""
  bs = BeautifulSoup(inf, 'html.parser')
  aux = bs.find_all('p')
  paragraphs = ''
  for x in aux:
    paragraphs += str(x)
  descArray = re.split('<p>|</p>|<br/>|\\xa0', paragraphs)
  desc = ''
  for x in descArray:
    desc  += x
  return desc

def processActivities(data):
  dictionary = word_tokenize(str(data))
  dictionary = [word for word in dictionary if word.isalpha()]
  dictionary = [word.lower() for word in dictionary]
  stopWords = set(stopwords.words('spanish'))
  dictionary = [word for word in dictionary if word not in stopWords]
  return dictionary

##Clase

In [ ]:
class Categories:
  def __init__(self, name, count):
    self.name = name
    self.count = count
    self.words = {}
    self.logarithm = {}
    self.finalSum = 0

  def get_name(self):
    return self.name

  def get_count(self):
    return self.count

  def get_words(self):
    return self.words

  def get_specific_word(self, word):
    return self.words[word]

  def get_logarithm(self, word):
    return self.logarithm[word]

  def get_finalSum(self):
    return self.finalSum

  def had_log(self, word):
    return word in self.logarithm

  def set_logarithm(self, word, result):
    self.logarithm[word] = result

  def had(self, word):
    return word in self.words

  def add_words(self, word):
    if word in self.words:
      self.words[word] += 1
    else:
      self.words[word] = 1

  def add_finalSum(self, logarithm):
    self.finalSum += logarithm

  def reset_finalSum(self):
    self.finalSum = 0

  def increment(self):
    self.count = self.count + 1

## Recogida de datos


In [ ]:
wb = openpyxl.load_workbook('activities.xlsx')
sheet = wb.active
data = []
i= 0
for row in sheet.iter_rows(min_row=2, values_only=True):
    info = row[9].split(' | ')
    desc = row[10]
    full_string = 'La actividad es ' + info[0]
    full_string += ', situada en ' + info[1]
    full_string += ', proporcionada por ' + info[2]
    full_string += '. La descripción es: ' + get_description(desc)
    data.append(full_string)

wb = openpyxl.load_workbook('CATEGORIAS.xlsx')
sheet = wb.active
catego = []
for row in sheet.iter_rows(min_row=1, max_col=1, values_only=True):
  catego.append(row[0])


## Prueba

In [ ]:
## @title
#dataToWork = []
#for i in range(len(catego)):
#  dataToWork.append(data[i])
#
#catego_series = pd.Series(catego)
#factCatego = catego_series.factorize()[0]
#
#df = pd.DataFrame({"Labels": factCatego, "text": dataToWork, "assignment_group": catego})
#print(df.head())

In [ ]:
## @title
#from spacy.lang.es.stop_words import STOP_WORDS as es_stop
#from io import StringIO
#import string
#from sklearn.model_selection import train_test_split
#from sklearn.svm import LinearSVC
#from sklearn.feature_extraction.text import TfidfVectorizer
#
#final_stopwords_list = list(es_stop)
#final_stopwords_list.append('\r\n')
#final_stopwords_list.append(string.punctuation)
#
#tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words=final_stopwords_list,max_features=5000)
#features = tfidf.fit_transform(df.text).toarray()
#labels = df.Labels

In [ ]:
## @title
#from sklearn.model_selection import train_test_split
#from sklearn.svm import LinearSVC
#
#model = LinearSVC()
#
#X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.2, random_state=0)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
# @title
#from sklearn import metrics
#
#unic_label_train = df.groupby(['assignment_group'])['assignment_group'].size()
#unic_label_train = unic_label_train.index.get_level_values(0).tolist()
#
#print(metrics.classification_report(y_test, y_pred,
#                                  target_names=unic_label_train))

In [ ]:
#file_alias = "<nombre de tu fichero>"
#path_to_data = f'tu/ruta/relativa/{file_alias}.csv'try:
#  df = pd.read_csv(path_to_data, low_memory=False)
#  except FileNotFoundError:
#    raise FileNotFoundError(f"File {file_alias} at path {path_to_data} doesn't exist.")

## Procesamiento de texto y creación del diccionario


In [ ]:


dictionary = processActivities(data)
dictionary = list(set(dictionary))
dictionary.sort()
with open('vocabulario.txt', 'w') as f:
  f.write("Palabras totales: " + str(len(dictionary)) + "\n")
  f.write("\n".join(dictionary))

In [ ]:
_leisureCat = Categories('OCIO', 0)
_sportsCat = Categories('DEPORTES', 0)
_toursCat = Categories('TOURS', 0)
_culturalCat = Categories('CULTURAL Y GASTRONÓMICO', 0)
_welfareCat = Categories('BIENESTAR', 0)
AllCategories = [_leisureCat, _sportsCat, _toursCat, _culturalCat, _welfareCat]
selectedCat = 0
for i in range(101):
  activity = processActivities(data[i])
  if catego[i] == AllCategories[0].get_name():
    #ocio
    selectedCat = 0

  elif catego[i] == AllCategories[1].get_name():
    #deportes
    selectedCat = 1

  elif catego[i] == AllCategories[2].get_name():
    #tours
    selectedCat = 2

  elif catego[i] == AllCategories[3].get_name():
    #cultural
    selectedCat = 3

  elif catego[i] == AllCategories[4].get_name():
    #bienestar
    selectedCat = 4

  #Incrementamos el numero de actividades de ocio que hay
  AllCategories[selectedCat].increment()
  #recorremos las palabras claves de la descripcion de la actividad
  for j in range(len(activity)):
    AllCategories[selectedCat].add_words(activity[j])

  #else:
    #nada

##Creacion de cada fichero


In [ ]:
for i in range(len(AllCategories)):
  name = AllCategories[i].get_name() + ".txt"
  with open(name, 'w') as n:
    n.write('Número de documentos (actividades) del corpus: ' + str(AllCategories[i].get_count()) + '\n')
    n.write('Número de palabras del corpus: ' + str(len(AllCategories[i].get_words())) + '\n')
    for word in dictionary:
      if AllCategories[i].had(word):
        toLog = AllCategories[i].get_specific_word(word)
      else:
        toLog = 0
      logarithm = math.log((toLog + 1) / (len(AllCategories[i].get_words()) + len(dictionary)))
      AllCategories[i].set_logarithm(word, logarithm)
      n.write('Palabra: ' +  str(word) + ' Frec: ' +  str(toLog) + ' LogProb: ' + str(logarithm) + '\n')

#**Clasificador**

##Lectura

In [ ]:
wb = openpyxl.load_workbook('activities.xlsx')
sheet = wb.active
data = []
i= 0
for row in sheet.iter_rows(min_row=2, values_only=True):
    info = row[9].split(' | ')
    desc = row[10]
    full_string = 'La actividad es ' + info[0]
    full_string += ', situada en ' + info[1]
    full_string += ', proporcionada por ' + info[2]
    full_string += '. La descripción es: ' + get_description(desc)
    data.append(full_string)

# porcentajes de preferencia
# mejoras: distancias, cultura , recomendacio vectorial 10


##Clasificacion

In [ ]:
result = ''
vector = ''
AllSums = [0, 0, 0, 0, 0]

for i in range(len(data)):
  activity = processActivities(data[i])
  fullDesc = data[i]
  for cat in range(len(AllCategories)):
    AllCategories[cat].reset_finalSum()
    for j in range(len(activity)):
      if AllCategories[cat].had_log(activity[j]):
        AllCategories[cat].add_finalSum(AllCategories[cat].get_logarithm(activity[j]))
    AllSums[cat] = AllCategories[cat].get_finalSum()

  for j in range(10):
    result += activity[j] + " "
  result += '... \n'
  for j in range(len(AllCategories)):
    result += AllCategories[j].get_name()
    result += ': '
    result += str(AllSums[j])
    result += '\n'
  index = AllSums.index(np.max(AllSums))            # calcular porcentaje sengun peso logaritmico  con medias y varianza
  result += AllCategories[index].get_name()
  result += '\n\n'                                  # Medida de similitud (coseno)
  vector += AllCategories[index].get_name()
  vector += '\n'

with open('classificationInfo.txt', 'w') as n:
  n.write(result)

with open('classification.txt', 'w') as n:
  n.write(vector)